# Requests

For downloading web content, Python features the [`requests`](https://docs.python-requests.org/en/master/user/quickstart/) library. MicroPython `urequests` implements a subset of this library. Let's download it.

In [7]:
%connect esp32
%upip -t internet/code/lib install urequests
%rsync

Connected to esp32 @ serial:///dev/ttyUSB0
Installing micropython-urequests 0.6 from https://micropython.org/pi/urequests/urequests-0.6.tar.gz
UPDATE  /lib/urequests.py


Fetch a sample webpage:

In [8]:
import urequests as requests

res = requests.get('https://micropython.org')
print(res.text)

# recycle the memory when downloading large pages
res = None

<!DOCTYPE html>



<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <!-- The above 3 meta tags *must* come first in the head -->

    <link rel="icon" href="/static/img/favicon.ico">

    <title>MicroPython - Python for microcontrollers</title>

    <link href="/static/bootstrap-3.3.7-dist/css/bootstrap.min.css" rel="stylesheet">

    <link href="/static/css/style.css?v=2" rel="stylesheet">

    
  <link href="/static/css/pygments.css" rel="stylesheet">
  <link href="/static/home/css/style.css" rel="stylesheet">


    <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700,800" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700" rel="stylesheet">

    <meta property="og:type" content="website" />
    <meta property="og:url" content="http://micropython.org/" />
    <meta property="og:title" cont

The output is rather verbose, with many links to images, etc. Great for humans, not so useful for computers.

Fortunately many sites offer special APIs with data formatted for computers. As an example, let's fetch weather data from https://openweathermap.org/. You'll need to first sign in and get an "API Key". I've added mine to `secrets.py`. 

Then consult the API (tab at the top of the page) for the correct formatting of the request. For example, to get current weather data, we send the following request:

In [8]:
import urequests as requests
import secrets

url = 'https://api.openweathermap.org/data/2.5/weather?q={},{},{}&appid={}'

res = requests.get(url.format(
    'Tahoe',
    'CA',
    'US',
    secrets.openweathermap_apiid
))

print(res.text)

{"coord":{"lon":-115.8071,"lat":46.0963},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01d"}],"base":"stations","main":{"temp":303.62,"feels_like":301.65,"temp_min":302.21,"temp_max":305.08,"pressure":1013,"humidity":13,"sea_level":1013,"grnd_level":905},"visibility":10000,"wind":{"speed":1.21,"deg":4,"gust":1.67},"clouds":{"all":0},"dt":1627179321,"sys":{"type":2,"id":2039021,"country":"US","sunrise":1627128986,"sunset":1627183364},"timezone":-25200,"id":5599665,"name":"Tahoe","cod":200}


The response is formatted in [json](https://www.json.org/json-en.html) and can easily be parsed with Python. In fact, json decoding is built right into the requests libray. Let's read and format the output:

In [8]:
j = res.json()
for k,v in j.items():
    # format and truncate long lines ...
    print("{:10}   {}".format(k, str(v)[:60]))

timezone     -25200
cod          200
dt           1627179321
base         stations
weather      [{'id': 800, 'icon': '01d', 'main': 'Clear', 'description': 
sys          {'country': 'US', 'sunrise': 1627128986, 'sunset': 162718336
name         Tahoe
clouds       {'all': 0}
coord        {'lon': -115.8071, 'lat': 46.0963}
visibility   10000
wind         {'gust': 1.67, 'speed': 1.21, 'deg': 4}
id           5599665
main         {'feels_like': 301.65, 'pressure': 1013, 'temp_max': 305.08,


The weather information is stored in `main`. Let's display just that:

In [8]:
print("Weather in {}".format(j['name']))
for k,v in j['main'].items():
    print("  {:10}   {}".format(k, v))

Weather in Tahoe
  feels_like   301.65
  pressure     1013
  temp_max     305.08
  temp         303.62
  temp_min     302.21
  humidity     13
  sea_level    1013
  grnd_level   905


Many websites offer APIs for communicating with computers. For example, [IFTTT](https://ifttt.com/) is site entirely organized around offering services for computer automation. 

Cloud service providers, like [IBM](https://www.ibm.com/cloud), [Microsoft Azure](https://azure.microsoft.com/), [Google Cloud](https://cloud.google.com/), [Amazon AWS](https://aws.amazon.com/) and many more offer web APIs. 

In fact, most websites and apps are backed by one of these services. Typically IoT devices, like ESP32's or Raspberry PIs, send data to the cloud service via one of these APIs. The data are analyzed and stored in the cloud and made available as web pages for human viewing.